In [17]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

In [18]:
with open("the-verdict.txt",'r',encoding="utf-8") as f:
    text = f.read()

enc_text= tokenizer.encode(text)

In [19]:
print(len(enc_text))

5145


In [20]:
enc_sample = enc_text[50:]

In [21]:
context_size = 4
# context size means how many words the model will look for predicting the next word in the sample
x  = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
# these are for example
print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


### for input 290 output will be 4920
### for input [290,4920] output will be 2241
### for input [290,4920,2241] output will be 287 .. and so on

In [22]:
#denoting the above statement
for i in range(1,context_size+1):
    input = enc_sample[:i]
    output = enc_sample[i]

    print(input, "---->", output)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [23]:
# showing it in actual words
for i in range(1,context_size+1):
    input = enc_sample[:i]
    output = enc_sample[i]

    print(tokenizer.decode(input), "---->", tokenizer.decode([output]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [24]:
from torch.utils.data import Dataset , DataLoader

In [25]:
import torch
class GPTdatasetV1(Dataset):
    def __init__(self,text,tokenizer,max_length,stride):
        self.input_ids = []
        self.output_ids =[]

        token_ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

        for i in range(0,len(token_ids)-max_length,stride):
            input_chunk = token_ids[i:i+max_length]
            output_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.output_ids.append(torch.tensor(output_chunk))
    
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self,idx):
        return self.input_ids[idx], self.output_ids[idx]

In [26]:
def create_dataloader_v1(text,batch_size=4,max_length = 256,stride =128,shuffle = True,drop_last = True,num_workers =0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTdatasetV1(text,tokenizer,max_length, stride)
    # creating dataloader
    dataloader = DataLoader(dataset=dataset,
                            batch_size=batch_size,
                            shuffle=shuffle,
                            drop_last=drop_last,
                            num_workers=num_workers)
    return dataloader

In [27]:
with open("the-verdict.txt",'r',encoding="utf-8") as f:
    raw_text = f.read()

In [30]:
print("PyToech Version:", torch.__version__)

dataLoader = create_dataloader_v1(text=raw_text,batch_size=1,max_length=4,stride=1,shuffle=False)
#creating a dataiterator for dataloader
data_iterator = iter(dataLoader)
first_batch = next(data_iterator)
print(first_batch)

PyToech Version: 2.5.1+cpu
[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [33]:
nex_batch = next(data_iterator)
print(nex_batch)
#with every execution the iterator will move forward and we will get a new output

[tensor([[1464, 1807, 3619,  402]]), tensor([[1807, 3619,  402,  271]])]


In [36]:
dataloader1 = create_dataloader_v1(raw_text,batch_size=8,max_length=5,stride=4,shuffle=False)
data_iter1 = iter(dataloader1)
input,target = next(data_iter1)
print("Input:\n",input)
print("\nOtput:\n",target)

Input:
 tensor([[   40,   367,  2885,  1464,  1807],
        [ 1807,  3619,   402,   271, 10899],
        [10899,  2138,   257,  7026, 15632],
        [15632,   438,  2016,   257,   922],
        [  922,  5891,  1576,   438,   568],
        [  568,   340,   373,   645,  1049],
        [ 1049,  5975,   284,   502,   284],
        [  284,  3285,   326,    11,   287]])

Otput:
 tensor([[  367,  2885,  1464,  1807,  3619],
        [ 3619,   402,   271, 10899,  2138],
        [ 2138,   257,  7026, 15632,   438],
        [  438,  2016,   257,   922,  5891],
        [ 5891,  1576,   438,   568,   340],
        [  340,   373,   645,  1049,  5975],
        [ 5975,   284,   502,   284,  3285],
        [ 3285,   326,    11,   287,   262]])
